# Setup

 Import standard modules

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
os.chdir('/home/chris/projects/tidal_flat_0d')

In [4]:
from src import features, models
from pyprojroot import here
root = here()
sys.path.append(str(root))
wdir = root / "data" / "interim"

Import project modules from src folder

In [5]:
tides = features.load_tide(wdir, "tides.feather")
tides = tides - (np.mean(tides) + 0.6)
# tides = tides.iloc[:int(len(tides) * 0.35)]

In [186]:
from src import models

sim = models.Simulation(
    tide_elev=tides.values,
    land_elev_init=1.15,
    index=np.arange(0, len(tides)),
    conc_init=0.0,
    conc_bound=1.0,
    grain_diam=3.5e-5,
    grain_dens=2.65e3,
    bulk_dens=900,
    org_rate=2e-4,
    comp_rate=4e-3,
    sub_rate=3e-3,
    slr=0.005,
    method="DOP853",
    runs=2,
    min_depth=0.0015,
    dense_output=False,
    save_inundations=True,
)
sim.simulate()

 48%|████▊     | 186.0/388.0 [02:03<03:55,  1.17s/inundation]

----------------------------------------
Starting elevation:        1.150      m
Final elevation:           1.178      m
Elevation change:          2.789     cm
----------------------------------------
Aggradation:               3.478     cm
Degradation:              -1.338     cm
----------------------------------------
Runtime:                      00:02:03


In [178]:
sim.one_run()

 48%|████▊     | 187.0/388.0 [18:13<14:43:07, 263.62s/inundation]

In [149]:
new_tides.index[0]

0

In [185]:
sim.inundations[-1].index

array([33354054, 33354055, 33354056, ..., 33356532, 33356533, 33356534])

In [180]:
sim.tides.pos

33356535

In [133]:
new_tides = models.Tides(
    index=sim.tides.index,
    tide_elev=sim.tides.tide_elev + sim.params.slr,
    land_elev=models.apply_rate(
        init_val=sim.results.data.land_elev.iloc[-1],
        index=sim.tides.index,
        timestep_sec=sim.params.timestep,
        rate_per_sec=sim.params.linear_rate_sec,
    ),
    min_depth=sim.params.min_depth,
)

In [ ]:
dense = sim.inundations[0].result.flood.sol(np.arange(sim.inundations[0].start_pos, sim.inundations[0].slack_pos, 1))

In [ ]:
available = dense[0] * dense[1]
extracted = dense[1] * settle_rate

In [ ]:
plt.plot(available >= extracted)

In [ ]:
sim.inundations[0].result.plot()

In [ ]:
DT = 1
fd_times = np.arange(0, len(tides), DT)
tide_fun = scipy.interpolate.InterpolatedUnivariateSpline(x=np.arange(0, len(tides)), y=tides.values)

slack, fd_result = aggrade_fd(
    times=fd_times,
    tide_fun=tide_fun,
    init_land_elev=init_land_elev,
    bound_conc=bound_conc,
    settle_rate=models.stokes_settling(grain_dia, grain_dens),
    bulk_dens=bulk_dens,
)
fd_aggradation = fd_result[3][-1] - fd_result[3][0]
"Percent Diff: {:.4f} %".format(abs(fd_aggradation - sim.results.aggradation) / (fd_aggradation + sim.results.aggradation) / 2 * 100)

In [ ]:
fig, axs = plt.subplots(3, 1)
fig.set_figheight(10)
fig.set_figwidth(15)
loc = np.where(fd_result[0] > 0.0015)[0][0]
i = fd_times[loc:] / 60
c = fd_result[2][loc:]
e = fd_result[3][loc:]


axs[0].plot(i, tide_fun(fd_times[loc:]), color="b")
axs[0].set(ylabel="Tide Elevation (m)")
axs[0].set(xticklabels=[])
axs[1].plot(i, c, color="r")
axs[1].set(ylabel="Concentration (g/L)",)
axs[1].set(xticklabels=[])
axs[2].plot(i, (e - fd_result[3][0])  * 1000, color="g")
axs[2].set(ylabel="Land Elevation $\Delta$ (mm)",xlabel="Time (min)")
for ax in axs:
    ax.axvline(slack / 60, color="k", linestyle="--")

In [ ]:
extracted

In [ ]:
integ = sim.inundations[0].result.flood
dt = np.diff(integ.t)
C = integ.y[1]
E = integ.y[2]
D = integ.y[0] - integ.y[2]
available = D[:-1] * C[:-1]
extracted = C[:-1] * dt * settle_rate
check = available >= extracted
available_per_step = (dt * available)
extracted_per_step = (dt * extracted)

In [ ]:
np.sum(extracted_per_step)

In [ ]:
(integ.y[2][-1] - integ.y[2][0]) * bulk_dens

In [ ]:
plt.plot(integ.t, C)

In [ ]:
C

In [ ]:
import time
def aggrade_fd(times, tide_fun, init_land_elev, bound_conc, settle_rate, bulk_dens,):
    start_time = time.perf_counter()
    land_elev = np.array([init_land_elev, init_land_elev])
    conc = np.array([0.0, 0.0])
    depth = np.array([tide_fun(0) - init_land_elev])
    status = 0
    index = np.arange(0, len(times))
    dt = times[1] - times[0]
    for i in index[1:-1]:
        t = times[i]
        depth = np.append(depth, tide_fun(t) - land_elev[i])
        if depth[i] < 0.0015:
            conc = np.append(conc, 0.0)
            land_elev = np.append(land_elev, land_elev[-1])
            continue

        if (depth[i] - depth[i-1]) > 0:
            delta_conc = - settle_rate * conc[i] / depth[i] * dt - 1 / depth[i] * (conc[i] - bound_conc) * (depth[i] - depth[i-1])
        elif (depth[i] - depth[i-1]) <= 0:
            if status == 0:
                slack = times[i]
                status = 1
            delta_conc = - settle_rate * conc[i] / depth[i] * dt
            
        if delta_conc < 0 and abs(delta_conc) > conc[i]:
            print("Extracted too much")
            delta_conc = - conc[i]
            
        conc = np.append(conc, conc[i] + delta_conc)
        delta_land_elev = settle_rate * conc[i] / bulk_dens * dt
        land_elev = np.append(land_elev, land_elev[i] + delta_land_elev)
        
    depth = np.append(depth, tide_fun(i+1) - land_elev[i])
    print("Aggradation:           {:.2e} m".format(land_elev[-1] - land_elev[0]))
    print("Degradation:           {:.2e} m".format(0.0))
    print("Elevation change:      {:.2e} m".format(land_elev[-1] - land_elev[0]))
    print("Final elevation:       {:.4f} m".format(land_elev[-1]))
    print("Runtime:               {}".format(time.strftime("%H:%M:%S", time.gmtime(time.perf_counter() - start_time))))

    return slack, np.stack((times, depth, conc, land_elev))